In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
pwd

'/content'

In [3]:
cd gdrive/My Drive/hoge

/content/gdrive/My Drive/hoge


In [4]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.applications import VGG16

Using TensorFlow backend.


In [0]:
classes = ['jui', 'ocha', 'piza','pop','pre','toppo']
num_classes = len(classes)
IMAGE_SIZE = 120

In [0]:
X_train, X_test, y_train, y_test = np.load('six_j.npy', allow_pickle=True)

In [0]:
# convert one-hot vector
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [0]:
# normalization
X_train = X_train.astype('float') / 255.0
X_test = X_test.astype('float') / 255.0

In [10]:
# include_top ; Whether to read last layer
# -> Add your own fully conected layer
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
vgg16_model.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120, 120, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
______________________________________________________________

In [11]:
# add 2 top layers
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))
top_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1179904   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 1,181,446
Trainable params: 1,181,446
Non-trainable params: 0
_________________________________________________________________


In [12]:
# combine models
model = Model(inputs=vgg16_model.input, outputs=top_model(vgg16_model.output))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120, 120, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 30, 30, 128)       0   

In [0]:
# Fix the weight just before the last conv layer of VGG16
for layer in model.layers[:15]:
    layer.trainable = False

In [14]:
opt = SGD(lr=1e-4, momentum=0.9)
# default : lr=0.001
# opt = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
6750/6750 [==============================] - 33s 5ms/step - loss: 0.4527 - accuracy: 0.8616
Epoch 2/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0329 - accuracy: 0.9976
Epoch 3/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0136 - accuracy: 0.9996
Epoch 4/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0088 - accuracy: 0.9997
Epoch 5/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0063 - accuracy: 0.9997
Epoch 6/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 7/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0039 - accuracy: 0.9999
Epoch 8/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0031 - accuracy: 0.9999
Epoch 9/10
6750/6750 [==============================] - 25s 4ms/step - loss: 0.0030 - accuracy: 0.9999
Epoch 10/10
6750/6750 [==============================] - 25s 4ms/step - l

In [15]:
score = model.evaluate(X_test, y_test, batch_size=32)
print('loss: {0} - acc: {1}'.format(score[0], score[1]))

2250/2250 [==============================] - 8s 3ms/step
loss: 0.0002516930972277704 - acc: 1.0


In [0]:
model.save('MMM.h5')